In [1]:
import requests

# Get API auth token

In [2]:
from getpass import getpass

username = "jochen"
password = getpass()

········


In [3]:
token_path = reverse("api-token-auth")
token_url = f"http://localhost:8000{token_path}"
token_url = f"https://python-podcast.staging.django-cast.com{token_path}"
print(token_url)

r = requests.post(token_url, data={"username": username, "password": password})
token = r.json()["token"]
print(token)

https://python-podcast.staging.django-cast.com/api/api-token-auth/
4acf2cddb07f66ceaf8bec25022ce4b2258fe5eb


# Get Training Data

In [4]:
path = reverse("cast:api:comment_training_data")
url = f"http://localhost:8000{path}"
url = f"https://python-podcast.staging.django-cast.com/api/comment_training_data/"
print(url)

https://python-podcast.staging.django-cast.com/api/comment_training_data/


In [5]:
%%time
headers = {"Authorization": f"Token {token}"}
r = requests.get(url, headers=headers)
messages = r.json()

CPU times: user 62.2 ms, sys: 9.48 ms, total: 71.7 ms
Wall time: 349 ms


In [6]:
len(messages)

1711

In [7]:
sf = SpamFilter.default
sf.retrain_from_scratch(messages)

In [8]:
sf.performance

{'ham': {'precision': 0.991304347826087,
  'recall': 0.9344262295081968,
  'f1': 0.9620253164556962},
 'spam': {'precision': 0.9949874686716792,
  'recall': 0.9993706733794839,
  'f1': 0.9971742543171115}}

# Evaluate

In [8]:
from cast.models.moderation import NaiveBayes, Evaluation

In [9]:
evaluator = Evaluation(model_class=NaiveBayes, num_folds=3)

In [10]:
evaluator.evaluate(messages)

{'ham': {'precision': 0.991304347826087,
  'recall': 0.9344262295081968,
  'f1': 0.9620253164556962},
 'spam': {'precision': 0.9949874686716792,
  'recall': 0.9993706733794839,
  'f1': 0.9971742543171115}}

# Train Spamfilter

In [7]:
from cast.models.moderation import NaiveBayes

In [81]:
import re
from collections import defaultdict

token_pattern = re.compile(r"(?u)\b\w\w+\b")
standard_tokenizer = token_pattern.findall


def regex_tokenize(message):
    return standard_tokenizer(message.lower())


def normalize(probabilities):
    try:
        factor = 1.0 / float(sum(probabilities.values()))
    except ZeroDivisionError:
        # not possible to scale -> skip
        return probabilities
    for name, value in probabilities.items():
        probabilities[name] *= factor
    return probabilities



class NaiveBayes:
    def __init__(self, tokenize=regex_tokenize, prior_probabilities={}, word_label_counts=None, threshold=0.5):
        self.tokenize = tokenize
        self.prior_probabilities = prior_probabilities
        if word_label_counts is None:
            self.word_label_counts = defaultdict(lambda: defaultdict(int))
        else:
            self.word_label_counts = word_label_counts
        self.number_of_words = self.get_number_of_words(self.word_label_counts)
        self.number_of_all_words = 0

    @staticmethod
    def get_label_counts(messages):
        label_counts = defaultdict(int)
        for label, text in messages:
            label_counts[label] += 1
        return label_counts

    def set_prior_probabilities(self, label_counts):
        number_of_messages = sum(label_counts.values())
        self.prior_probabilities = {label: count / number_of_messages for label, count in label_counts.items()}

    def set_word_label_counts(self, messages):
        counts = self.word_label_counts
        for label, text in messages:
            for word in self.tokenize(text):
                counts[word][label] += 1

    @staticmethod
    def get_number_of_words(word_label_counts):
        number_of_words = defaultdict(int)
        for word, counts in word_label_counts.items():
            for label, count in counts.items():
                number_of_words[label] += 1
        return number_of_words

    def fit(self, messages):
        self.set_prior_probabilities(self.get_label_counts(messages))
        self.set_word_label_counts(messages)
        self.number_of_words = self.get_number_of_words(self.word_label_counts)
        self.number_of_all_words = sum(self.number_of_words.values())
        return self

    @staticmethod
    def update_probabilities(probabilities, counts_per_label, number_of_all_words):
        updated_probabilities = {}
        for label, prior_probability in probabilities.items():
            word_count = counts_per_label.get(label, 0.5)
            word_probability = word_count / number_of_all_words
            updated_probabilities[label] = prior_probability * word_probability
        return updated_probabilities

    def predict(self, message):
        probabilities = dict(self.prior_probabilities)
        for word in self.tokenize(message):
            counts_per_label = self.word_label_counts.get(word, {})
            probabilities = normalize(self.update_probabilities(probabilities, counts_per_label, self.number_of_all_words))
        return probabilities

    def predict_label(self, message):
        probabilities = self.predict(message)
        if len(probabilities) == 0:
            return None
        return sorted(((prob, label) for label, prob in probabilities.items()), reverse=True)[0][1]

    def dict(self):
        return {
            "class": "NaiveBayes",
            "prior_probabilities": self.prior_probabilities,
            "word_label_counts": self.word_label_counts,
        }

    def __eq__(self, other):
        return (
            self.prior_probabilities == other.prior_probabilities and self.word_label_counts == other.word_label_counts
        )

# Split Train/Test

In [8]:
import random


def split_into_folds(items, folds):
    k, m = divmod(len(items), folds)
    return [items[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(folds)]


def flatten(items):
    return [item for sublist in items for item in sublist]



def generate_train_test(folds):
    for i, n in enumerate(folds):
        all_but_n = flatten(folds[:i]) + flatten(folds[i + 1:])
        yield n, all_but_n

## Split Into Spam and Ham

To be able to sample stratified.

In [9]:
spam, ham = [], []
for label, msg in messages:
    if label == "ham":
        ham.append((label, msg))
    else:
        spam.append((label, msg))
random.shuffle(spam)
random.shuffle(ham)
print(len(spam), len(ham))

1589 122


In [10]:
num_folds = 3
ham_folds = split_into_folds(ham, num_folds)
spam_folds = split_into_folds(spam, num_folds)
folds = []
for ham_fold, spam_fold in zip(ham_folds, spam_folds):
    folds.append(ham_fold + spam_fold)

In [11]:
for test, train in generate_train_test(folds):
    print(len(test), len(train))

571 1140
571 1140
569 1142


In [19]:
x = [[1, 2], [4, 5]]
flatten(x)

[1, 2, 4, 5]

# Evaluate

In [12]:
for test, train in generate_train_test(folds):
    print(len(test), len(train))
    nb = NaiveBayes().fit(train)

    true_positives = 0
    all_observations = len(test)

    for label, message in test:
        if nb.predict_label(message) == label:
            true_positives += 1

    accuracy = true_positives / all_observations
    print(f"Accuracy: {accuracy:.3f}", true_positives, all_observations)

571 1140
Accuracy: 1.000 571 571
571 1140
Accuracy: 0.988 564 571
569 1142
Accuracy: 0.996 567 569


In [110]:
for test, train in generate_train_test(folds):
    print(len(test), len(train))
    nb = NaiveBayes().fit(train)

    true_positives = 0
    all_observations = len(test)

    for label, message in test:
        if nb.predict_label(message) == label:
            true_positives += 1

    accuracy = true_positives / all_observations
    print(f"Accuracy: {accuracy:.3f}", true_positives, all_observations)

571 1140
Accuracy: 0.802 458 571
571 1140
Accuracy: 0.809 462 571
569 1142
Accuracy: 0.772 439 569


In [13]:
def generate_outcomes(nb, test_messages):
    outcomes = (("true", "false"), ("positive", "negative"))
    possible_results = [f"{a}_{b}" for b in outcomes[1] for a in outcomes[0]]
    result_template = dict.fromkeys(possible_results, 0)

    labels = set(nb.prior_probabilities)
    label_results = {label: dict(result_template) for label in labels}
    all_observations = len(test_messages)

    for label, message in test_messages:
        predicted = nb.predict_label(message)
        if label == predicted:
            label_results[label]["true_positive"] += 1
        else:
            label_results[label]["false_negative"] += 1
            label_results[predicted]["false_positive"] += 1
    return label_results

In [14]:
def precision_recall_f1(result):
    all_observations = sum(result.values())
    tp = result["true_positive"]
    fp = result["false_positive"]
    fn = result["false_negative"]
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    return precision, recall, f1


def show_result(label_results):
    for label, result in label_results.items():
        precision, recall, f1 = precision_recall_f1(result)
        print(
            f"{label: >4} f1: {f1:.3f} precision: {precision:.3f} recall: {recall:.3f}"
        )

In [15]:
results = None
for test, train in generate_train_test(folds):
    print(len(test), len(train))
    nb = NaiveBayes().fit(train)
    label_result = generate_outcomes(nb, test)
    if results is None:
        results = label_result
    else:
        for label, counts in label_result.items():
            for name, count in counts.items():
                results[label][name] += count
show_result(results)

571 1140
571 1140
569 1142
spam f1: 0.997 precision: 0.995 recall: 0.999
 ham f1: 0.962 precision: 0.991 recall: 0.934


In [16]:
results = None
for test, train in generate_train_test(folds):
    print(len(test), len(train))
    nb = NaiveBayes().fit(train)
    label_result = generate_outcomes(nb, test)
    if results is None:
        results = label_result
    else:
        for label, counts in label_result.items():
            for name, count in counts.items():
                results[label][name] += count
show_result(results)

571 1140
571 1140
569 1142
spam f1: 0.997 precision: 0.995 recall: 0.999
 ham f1: 0.962 precision: 0.991 recall: 0.934


In [17]:
nb.prior_probabilities

{'ham': 0.07180385288966724, 'spam': 0.9281961471103327}

In [73]:
for num, (label, message) in enumerate(messages):
    if label != nb.predict_label(message):
        print(num, nb.predict(message))
        print(message)
        break

234 {'ham': 3.089224154969338e-05, 'spam': 0.9999691077584503}
DjangoCon Europe 2021  Johannes und Jochen waren auf der DjangoCon Europe 2021 und erzählen Dominik davon. Beispielsweise, weshalb vielleicht keine so gute Idee ist, zuviel Spaß beim Programmieren zu haben. Oder welche Talks und Workshops besonders interessant, gut oder einfach nur überraschend waren.




  
 
Shownotes

Unsere E-Mail für Fragen, Anregungen & Kommentare: hallo@python-podcast.de

DjangoCon Europe 2021


	DjangoCon Europe 2021 
	Talk: Programming for pleasure | What nobody tells you about documentation
	ATEM Mini
	Talk: Serving files with Django, django_fileresponse
	nginx X-Accel | ngx_http_auth_request
	CDN
	Django 3.1 Async | Django wird asynchron: Pythons Web-Framework erhält neue Funktion
	MinIO
	Jochens Twitch Stream | Youtube Playlist
	Talk: Django Unstuck: Suggestions for common challenges in your projects | Video und Material zu Django Unstuck
	DjangoCon 2020 | How To Get On This Stage (And What To D

In [56]:
nb.prior_probabilities

{'ham': 0.09710391822827939, 'spam': 0.9028960817717206}

In [62]:
sum(nb.number_of_words.values())

24165

In [59]:
0.5 / 20850, 0.5 / 3315

(2.398081534772182e-05, 0.00015082956259426848)

In [74]:
message = messages[2][1]
probabilities = dict(nb.prior_probabilities)
print(probabilities)
for word in nb.tokenize(message):
    counts_per_label = nb.word_label_counts.get(word, {})
    print(word, counts_per_label)
    probabilities = normalize(nb.update_probabilities(probabilities, counts_per_label, nb.number_of_words))
    print(probabilities)

{'ham': 0.09710391822827939, 'spam': 0.9028960817717206}
joshuagrere {}
{'ham': 0.09710391822827938, 'spam': 0.9028960817717205}
uliseskodylgxa {}
{'ham': 0.09710391822827938, 'spam': 0.9028960817717205}
gmail defaultdict(<class 'int'>, {'ham': 1, 'spam': 296})
{'ham': 0.0003632030687473317, 'spam': 0.9996367969312526}
com defaultdict(<class 'int'>, {'ham': 15, 'spam': 2199})
{'ham': 2.4784047247959827e-06, 'spam': 0.9999975215952751}
https defaultdict(<class 'int'>, {'ham': 17, 'spam': 2651})
{'ham': 1.5893241823647853e-08, 'spam': 0.9999999841067582}
zootovaryvsems defaultdict(<class 'int'>, {'spam': 1})
{'ham': 7.94662097497271e-09, 'spam': 0.999999992053379}
site defaultdict(<class 'int'>, {'spam': 378})
{'ham': 1.0511403489468207e-11, 'spam': 0.9999999999894885}
post defaultdict(<class 'int'>, {'spam': 27})
{'ham': 1.946556201773454e-13, 'spam': 0.9999999999998054}
cat defaultdict(<class 'int'>, {'spam': 5, 'ham': 1})
{'ham': 3.893112403547514e-14, 'spam': 0.9999999999999611}
chow

In [51]:
nb.tokenize(messages[2][1])

['joshuagrere',
 'uliseskodylgxa',
 'gmail',
 'com',
 'https',
 'zootovaryvsems',
 'site',
 'post',
 'cat',
 'chow',
 'urinary',
 'https',
 'ic',
 'pics',
 'livejournal',
 'com',
 'shalenaolena',
 '21982935',
 '37303',
 '37303_900',
 'jpg',
 'руны',
 'дариус',
 'люкс',
 'руна',
 'руны',
 'секс',
 'кейл',
 'руны',
 'руна',
 'стан',
 'раскинуть',
 'руны',
 'руна',
 'кайса',
 'руна',
 'квеорт',
 'руне',
 'белсвик',
 'камилла',
 'руны',
 'href',
 'https',
 'zootovaryvsems',
 'site',
 'post',
 'cat',
 'chow',
 'urinary',
 'img',
 'src',
 'https',
 'ic',
 'pics',
 'livejournal',
 'com',
 'shalenaolena',
 '21982935',
 '37303',
 '37303_900',
 'jpg',
 'width',
 '400',
 'height',
 '400',
 'alt',
 'руны',
 'https',
 'zootovaryvsems',
 'site',
 'post',
 'cat',
 'chow',
 'urinary',
 'седьмая',
 'руна',
 'тимо',
 'руны',
 'руны',
 'иллаой',
 'леона',
 'руны',
 'гномьи',
 'руны',
 'руны',
 'рода',
 'руны',
 'вуньо',
 'руны',
 'фидл',
 'руны',
 'исцеления',
 'лостфильм',
 'рун',
 'рун',
 'флат',
 'hi'

In [42]:
messages[1543][1]

"HTMX Heute geht es um ein zur Zeit ganz heisses Thema: HTMX. Vielleicht braucht ja nicht jede Webseite eine SPA zu sein?\xa0Thomas\xa0hat sowohl auf der DjangoCon Europe wie US einen Vortrag über htmx gehalten und daher unterhalten sich\xa0Dominik und Jochen\xa0heute auch mit ihm darüber :).\n\n\xa0 \n\n\n \n\xa0\n\nDas Datenformat, an das sich Jochen nicht mehr erinnern konnte, war übrigens Message pack.\n\nShownotes\n\nUnsere E-Mail für Fragen, Anregungen & Kommentare: hallo@python-podcast.de\n\nNews aus der Szene / Programmierpodcasts\n\n\n\tPython 3.6 End of Life\n\tTIOBE Index for January 2022\xa0(das Datum ist nicht in der URL, wtf)\n\tprogrammier.bar\n\tWorking Draft\n\tSoftware Architektur im Stream\n\tINNOQ Podcast\n\tWO WiR SIND IST VORNE.\n\tTechtiefen\n\tKI in der Industrie\n\n\nWerbung\n\n\n\tNordVPN 2-Jahres-Paket + 1 Monat gratis\n\n\nHTMX\n\n\n\thtmx\n\tREST\xa0/\xa0Architectural Styles and the Design of Network-based Software Architectures\xa0Dissertation von Roy Fiel

In [98]:
count = 0
for label, message in messages:
    if label == "ham":
        print(message)
        count += 1
print(count)

Jürgen  Nach drei Wochen Pendeln hatte ich die Folge durch 😅

Nachdem ich in den letzten Jahren fast nur mit ZODB gearbeitet habe, habe ich durch die Folge viel Neues gelernt! Vor allem die Geschichte mit Postgres und JSON hört sich ja super an.

Vielen Dank und immer weiter so!
Norbert  Danke für diese sehr interessante Folge!

Für mich waren viele nutzbare Infos dabei, die ich entweder noch gar nicht wusste oder jedenfalls nicht derart detailliert.

Gruss
Norbert  War interessant für mich, danke!
Jürgen  DNS over https im Firefox kommt afaik nur in den USA.
Johannes  Voll die gute Episode!
Jochen  Danke :)
Jochen  Nee, ist gut. Hilft dabei uns dazu zu bringen, dass das hier mal weitergeht :).
Jochen Wersdörfer  Also ich mache das vor allem auch, um selbst etwas dabei über das Thema zu lernen. Und öffentlich failen hat halt einen ganz guten Lerneffekt :). Aber ja, vielleicht sollten wir uns zumindest bei Themen, mit denen wir uns nicht eh schon jeden Tag beschäftigen etwas besser vorb

Ajo, wieder eine klasse Episode! Hoffe bald auf Nachschub!
Alpengreis  Wieder einiges gelernt, danke!
Jochen  Oh, wenn du eine gute Hostinglösung gefunden hast, sag bescheid :).

Hmm, jo - also zum Verschieben von Dateien nehme ich meistens scp/rsync. Wofür möchtest du den Dateien zwischen Rechnern verteilen? Vielleicht gibt es für deinen Anwendungsfall ja auch noch andere Möglichkeiten...
Dominik  oh, jetzt habe ich aus Versehen einen Kommentar gelöscht der sich auf die letzte Folge bezog.. in dem stand: "*piep, *piep, *piep"
da ging sofort mein Spam-Filter an :-----P
Jochen Wersdörfer  Hallo Philip,

vielen Dank für deinen Hinweis. Ja, Adressen eine normalisierte Form zu bringen, ist irgendwie immer schwieriger als man denkt. Je nachdem wofür man den Ort verwenden möchte, kann man das vielleicht  schon so machen, denn vielleicht braucht man bei einer Marketingkampagne gar nicht den Ortsnamen, sondern halt nur die ungefähre Region - aber es hängt halt immer davon ab, was der Zweck der

Da braucht man die Klasse natürlich für. Danke auch :).
Jochen Wersdörfer  Vielen Dank - ja, waren einige ausführliche Abschweifungen dabei diesmal. Aber es gab halt auch viel zu erzählen 😇.

Dein Kommentar war übrigens auch der erste, der nach dem Umstieg auf wagtail nicht vom Spamfilter abgelehnt wurde. Gut, dass das mal jemand getestet hat 😅.
Johannes  Stimmt, da haben wir einen Fehler gemacht. Danke für das Ausprobieren und die Korrektur!
Frank Mankel  Hallo Podcast Team,

ich finde es absolut rekordverdächtig, Euer Intro hat diesmal fast 1 1/2 Stunden gedauert. Meinen Respekt dafür ;)

Spaß beiseite, ich mag Euren Podcast und freue mich auf jede Folge. Ok, ab und zu bin ich von den Themen überfordert (Python Einsteiger) aber ich habe schon einiges bei Euch gelernt.

Danke für den tollen Podcast!

Gruß
FrankM
Jochen Wersdörfer  Schön zu hören :).
Alpengreis  Ah ja, eine EDIT-Funktion für die Beiträge hier wäre auch noch schön :-)
Frank  ... bei dem Gestammel zu Vue habe ich dann ab

Norbert
Dominik  Hey Dirk, Danke für die Links und sorry, für die verspätete Antwort - hier geht es gerade drunter und drüber.
Besonders die letzte der Definitionen von DevOps ist sehr interessant - es entwickelt & verflechtet sich quasi gemeinsam ineinander.
Hatten die Links ein Ablaufdatum? - letzte Woche konnte ich sie noch sehen, heute sind sie nicht mehr verfügbar. Ich fände es als "anderer Leser" total spannend, wenn ich die Links hier ebenfalls angucken könnte.

Ich schaffe es leider nicht auf die FrOSCon, Jochen war afaik auf "vielleicht"

Gruß,
Dominik
Domi  *Hüsterchen https://www.youtube.com/watch?v=WVDQEoe6ZWY
Alpengreis  Ja, Du hast schon recht, dass mit dem gleichen Integer reproduzierbare Ergebnisse "forciert" werden, wie z.B. mit:

import random

for i in range(2):
    random.seed(2020)
    for i in range(20):
        print(random.random())
    print()


Im Gegensatz zu Folgendem:

import random

for i in range(2):
    random.seed()
    for i in range(20):
        print